# Web Crawling as a Reinforcement Learning task

The task we'll be solving is how to find web forms on all kinds on a web site. Crawler should find a login form, registration form, search form, order form, etc., making as few requests as possible.

With an appropriate stopping criteria (e.g. stop either when all form types are found or a certain number of requests is sent) a web site (a domain) can be seen as an episode in RL terms. We don't have to use website-specific request limits though, so the task is not necessarily split into episodes.

For a web crawler it is much more efficient to process several domains in parallel, because often we're limited by a website speed. This is different from a traditional RL setting where all episodes are processed sequentially.

For a RL task we should define what are states and what are actions. For web crawling there are many ways to define actions; a natural way is to use URL in a frontier as an action. In literature other actions are used, e.g. whether to follow all links from a given page or not. **Here we'll use crawl frontier as a set of actions.**

State is some information about the crawl so far. Sometimes last web page is used as state; this is not optimal because crawler may choose to follow any link from a frontier, not necessarily a link from the last web page.  Sometimes state is discarded. Sometimes links are considered states (in this case action is whether to follow a given link or not). 

For our task **information about the current domain** is the state: how many forms are found so far, how many requests are sent to this domain (what else?).

## Notation

* $s_t$ - state; information about a web site at time step $t$;
* $a_t$ - action; a link we choose to follow at time step $t$;
* $s_{t+1}$ - updated web site information after we followed link $a_t$.
* $r_{t+1}$ - reward; a number of new forms we find after following link $a_t$. We observe $s_t$ and $a_t$ before making a HTTP request, then we observe $r_{t+1}$ and $s_{t+1}$ after receiving a response. Be careful: often in literature instead of $r_{t+1}$ people write $r_t$, likely because $r_t$ is seen as a reward for action $a_t$. For crawling task $r_{t+1}$ notation looks more natural.
* $R_t$ - return; $R_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + ...$ - discounted number of new forms we will find after time step $t$ if we'll be following our policy $\pi$ (?). We don't know this value until the end of an episode (or we won't know its true value ever if the problem is not split into episodes). The sum is discounted because it is better to find new forms sooner, using a least amount of requests. $0 \leq \gamma \leq 1$ parameter controls how important is it to get results sooner. When $\gamma = 1$ there is no discounting; in this case it should be required that tasks are split into finite episodes because return can diverge (?).
* $Q(s, a) = E_{\pi}(R_t | s_t=s, a_t=a)$ - expected future return when we start at state $s$ and follow link $a$. In other words, $Q(s, a)$ is an estimate of a discounted number of new forms we'll find for the website $s$ if we follow link $a$ and then keep following links according to the control policy $\pi$. This is the function we're going to learn. It can be used to solve control problem because it provides scores which are larger for more promising links. This function has $\gamma$ as a hyperparameter; if $\gamma=0$ then $Q(s, a)$ is an expected number of new forms we'll find on a page $a$ links to. 

## What web site to crawl next
Processing multiple domains in parallel poses extra complexity: we need to select best link not only for a single domain, but for all domains. The best link is not necessarily a link with a highest $Q(s, a)$ score: there may be other priorities. For example, it may be desirable to explore many websites, not to be stuck in a single domain.

In order to account for this we provide a separate requests priority queue for each website. To select next link crawler first chooses a domain, then chooses a link for this domain. At each time step the domain is selected randomly; the probability of selecting a domain depends on a maximum score of all its links. This allows to select all domains, but to select promising domains more often. 

## What link to follow

To prioritize requests for a domain we use $Q(s, a)$ function. As crawler crawls it learns a better approximation of $Q(s, a)$ function; this allows to select better links.

Because state and action space is very large we use function approximation for $Q(s, a)$: first, information about $s$ and $a$ is converted to a feature matrix, and then a regression model is used to compute $Q(s, a)$; it could be a linear regression model or a neural network. 

To train the regression model true values are needed. Recall that $Q(s, a)$ is an expected value of $R$. So one way to train $Q(s, a)$ model would be to wait until $R$ value is known: crawl the domain in full, then compute returns for each state/action pair and use them as a training data. This has several issues: crawling domain in full has a high cost and it is not necessary in most cases; it is desirable to update crawler online based on partial observed information. We're processing several domains in parallel, this would have made such 'offline' updates even worse.

To update $Q(s, a)$ without knowing true $R_t$ values we can use their estimates. Recall that $R_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + ...$ . After HTTP response is received we know true $r_{t+1}$ value (a number of new forms) and updated information about the domain $s_{t+1}$; to estimate $r_{t+2}$, $r_{t+3}$ and the rest of terms we can use current $Q(s, a)$ function: $R_t \approx r_{t+1} + \gamma Q(s_{t+1}, a_{t+1})$. To do that we need to choose $a_{t+1}$ - which link is expected to be followed. There are two main options:

1. "Q-learning": select a link with top $Q(s, a)$ score, not necessarily a link we'll follow next.
2. "SARSA": select a link which we'll crawl next, not necessarily a link with top $Q(s, a)$ score.

It is useful not to choose only links with top $Q(s, a)$ scores to allow crawler to explore more options. A common approach is using an $\epsilon$-greedy policy: an action with top score is selected with probability $1-\epsilon$; a random action is selected with probability $\epsilon$. $0 \leq \epsilon \leq 1$ is a parameter which controls exploration-exploitation tradeoff; with $\epsilon=0$ crawler is greedy; with $\epsilon=1$ crawler only follows random lnks.

Q-learning allows to learn an optimal policy even if current crawling policy is not optimal; it is called an off-policy method because policy learned is not the same as the policy crawler is using. SARSA is an on-policy method; it learns $Q(s, a)$ for the same policy as used for crawling. The return of a crawler which uses SARSA is likely to be higher: crawler can make better decisions because $Q(s, a)$ accounts for $\epsilon$-greedy behaviour. With $\epsilon \rightarrow 0$ both SARSA and Q-learning should converge to the same policy, but with Q-learning we can learn an optimal policy even with non-zero $\epsilon$.

Following standard Q-learning algorithm we'd have to choose $a_{t+1}$ link (used for $R_t$ estimate) from all links in the same domain, but not necessarily from a page we've just downloaded. For web crawling this approach doesn't work well. Let's say we've just followed a link from page $A$ to page $B$ and there is no reward at page $B$, and page $B$ doesn't have promising links on it. It doesn't make sense to prioritize links like $A \rightarrow B$, but this is what will happen if there is a high-priority link in a frontier. There can be high-priority links because of many reasons: crawler can be optimistic and assing all links high priority, crawler may be exploring accoring to $\epsilon$-greedy strategy, crawler may be solving a different task. 

Instead of standard Q-learning or SARSA it makes sense to use a modified strategy: $R_t$ estimates use only links $a$ from the page the crawler just downloaded.

So, after response is received we can get an estimate of $R_t \approx r_{t+1} + \gamma Q(s_{t+1}, a_{t+1})$. It means we can train a regression model which minimises squared difference between observed $R_t$ value (i.e. its estimate $r_{t+1} + \gamma Q(s_{t+1}, a_{t+1})$ and predicted $R_t$ value (i.e. $Q(s_t, a_t)$).

## Updating request priorities

Computing $Q(s, a)$ for all links in a frontier and updating priority queues at each time step is too costly. 
$Q(s, a)$ function is updated after each response, but priorities of already queued requests are updated only from time to time (e.g. every 30s or after every 500 responses). Priorities for new requests (for links extracted from a just received page) are computed using up-to-date $Q(s, a)$ function.

$Q(s, a)$ values depend on state $s$ heavily; this means at each time where state $s$ is updated we also have to change all link priorities. Recall that domain state currently consists of (a) number of forms found for each form type and (b) 'domain time', a number of requests sent so far. As an optimization, all priorities are updated only when (a) changes, not when (b) changes.

## Crawl Graph

The input to crawler is a list of seed URLs; they are assumed to be to different websites. Initially, crawler makes requests to a batch of URLs from a seed list and waits for responses. 

To update $Q(s,a)$ function crawler needs:

1. $s_t$ - information about the current domain;
2. $a_t$ - information about the link which lead to the last received response;
3. $r_{t+1}$ - observed reward (a number of new forms on a page);
4. $s_{t+1}$ - updated information about the domain; we can compute it based on last received response;
5. $a_{t+1}$ - next link to follow (in Q-learning)

## Solving Multi-Task Learning Problem

The goal of a crawler is to find forms of all types detected by Formasaurus. It means it is solving several tasks at once. For example, some links may be more promoising for a task of finding login forms, while other can be more promising if we want to find search forms.

There are two ways to deal with that:

1. learn separate Q functions for each task and then choose what goal to pursue;
2. learn a single Q function which solves all task at the same time; reward can be sum of rewards of individual tasks.

(2) is easier to implement. (1) allows to use individual policies after training.
So far (2) is implemented.

For (1) we need to choose which task to pursue. In other words, first we select $Q$ function and then select best action $a$ according to this function. $Q$ funciton can be selected randomly, or in a round-robin fashion, or we can use a more elaborate policy for that.

Because there are several $Q$ functions each request should have several priorities.

## Exploration: $\epsilon$-greedy policy

In order for crawler to explore it can sometimes make a random request; this is called $\epsilon$-greedy policy. $1 - \epsilon$ is a probability of crawler being greedy (i.e. choosing most promising request).

Recall that we have to (a) select domain to crawl and (b) select a link from this domain to crawl. There are two options:

1. use $\epsilon$-greedy policy for both (a) and (b);
2. use $\epsilon$-greedy policy only for (b).

In Deep-Deep crawler $\epsilon$-greedy policy is used both for (a) and (b).

## Naive Implementation: Results

A naive version of the crawler can be implemented as follows:

1. State is a vector with a number of forms found so far for each form type.
2. Action is a vector of features extracted from links (url, text).
3. Q function is approximated by a linear regression model. The model is trained using SGD with a constant step. Features are concatenated state and action vectors.

As it turns out, naive implementation of Q-learning doesn't work well. It looks like a Linear Regression model is not powerful enough to handle this kind of features. 

For example, let's say we've already found a login form for this domain, and there is an unvisited link with 'login' in it. Regression model should learn that return for this link should be low. To do that it can assign negative weight for state:login feature and positive weight for url-has:login feature. But by assigning a negative weight for state:login we'll decrease priorities of all other links as well, not only of login links. Model will try to adjust weights of other featuers to account for this, but it is a band-aid.

## TODO: Experience Replay

Experience replay is a method of making training more stable. The main idea is to use random examples from the past in addition to the most recent example to update parameters of $Q(s,a)$ function. By crawling multiple domains in parallel we avoid some of the issues of incremental $Q(s_t, a_t)$ updates. But even in this case experience replay turns out to be helpful. 

## TODO: Looking more than one step ahead

The most straightforward way to learn $Q(s, a)$ function which looks more than 1 step ahead is to use a different approximation of $R_t$. Instead of $R_t \approx r_{t+1} + \gamma Q(s_{t+1}, a_{t+1})$ we can use $R_t \approx r_{t+1} + \gamma r_{t+2} + \gamma^2 Q(s_{t+2}, a_{t+2})$ or a weighted sum of these approximations.